In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

In [13]:
data = pd.read_csv('Wholesaledata.csv')
print(data.head())

   Channel  Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen
0        2       3  12669  9656     7561     214              2674        1338
1        2       3   7057  9810     9568    1762              3293        1776
2        2       3   6353  8808     7684    2405              3516        7844
3        1       3  13265  1196     4221    6404               507        1788
4        2       3  22615  5410     7198    3915              1777        5185


In [14]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=["Total", "Percent"])
missing_data.head(20)

,Total,Percent
Delicassen,0,0.0
Detergents_Paper,0,0.0
Frozen,0,0.0
Grocery,0,0.0
Milk,0,0.0
Fresh,0,0.0
Region,0,0.0
Channel,0,0.0


In [15]:
print("Channel: ",data.Channel.unique())

Channel:  [2 1]


In [16]:
print("Region: ",data.Region.unique())

Region:  [3 1 2]


In [17]:


    # drop last column (extra column added by pd)
    # and unnecessary first column (id)
data.drop(data.columns[[0]], axis=1, inplace=True)
    
    #Mapping values of target variable quality to 'low', 'medium' and 'high' categories for classification
data['Region']=data['Region'].map({1:'low',2:'medium',3:'high'})  
    #df['quality']=df['quality'].map({'low':0,'medium':1,'high':2})
    
    
diag_map = {'low': 1.0, 'medium': -1.0,'high':-1.0}
data['Region'] = data['Region'].map(diag_map)
    
    
    
    # put features & outputs in different data frames
Y = data.loc[:, 'Region']
X = data.iloc[:, 0:-1]

In [18]:
import numpy as np
from itertools import combinations_with_replacement
from sympy.core import Mul
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

In [19]:
def Multiply(a):
    ans = 1
    for i in a:
        ans*=i
    return ans

In [20]:
def Create_features(variables):
    ans = []
    for i in range(1,5):
        l = list(combinations_with_replacement(variables, i))
        for j in l:
            ans.append(Multiply(j))
            #print(Multiply(j))
    ans.append(1)
    ans = np.array(ans).astype(float)
    return ans.reshape((ans.shape[0],1)) 

In [21]:
def smo(X,Y, C=0.1, toler=0.001, maxIter=5):
    #dataMatrix = mat(dataMatIn); 
    #labelMat = mat(classLabels).transpose()
    b = 0; m = X.shape[0]
    alpha=np.zeros(shape=(X.shape[0],1))
    #alphas = mat(zeros((m,1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m): #Fetch one piece of data each time, corresponding to alpha1
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b #Calculate the classification function result, directly apply the classification function formula (8)
            Ei = fXi-float(labelMat[i])#if checks if an example violates KKT conditions #The difference between the calculated result and the actual classification
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                               #3 cabinets that do not meet the KKT condition in the iteration are used here, ((labelMat[i]*Ei> toler) and (alphas[i]> 0) as an example, assuming this is a positive classification, then (labelMat[i ]*Ei> toler)
                               # Ei>0 and Ei> toler, because labelMat[i]=1, yi*fXi>1 is obtained, at this time, alpha1>0 is not satisfied, and the original alpha1=0
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b # alpha2 random selection
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]): # y1 and y1 have different signs, calculate L and H
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: 
                    print ("L==H") 
                    continue
                    eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:]. T #Calculate η
                if eta >= 0: 
                    print ("eta>=0") 
                    continue #When calculating η =0, divide by 0, skip this case temporarily
                    alphas[j] -= labelMat[j]*(Ei-Ej)/eta #Calculate alpha2, refer to formula (5)
                    alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j]-alphaJold) <0.00001):
                    print ("j not moving enough")
                    continue #reach the optimization threshold
                    alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])#update i by the same amount as j
                                                                                                                                                 #the update is in the oppostie direction Update alpha1, formula (6)
                                 #Calculate b1,b2, formula (7)                                                        
                    b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                    b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                    if (0 < alphas[i]) and (C > alphas[i]):
                        b = b1
                    elif (0 < alphas[j]) and (C > alphas[j]): 
                        b = b2
                    else: 
                        b = (b1 + b2)/2.0
                        alphaPairsChanged += 1
                        print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
                if (alphaPairsChanged == 0): 
                    iter += 1
                else: 
                    iter = 0
        print ("iteration number: %d" % iter)
         
        return b,alphas
    

In [22]:
def predict(X,Y,alpha,b,x,sigma):
    result=0.0

    for i in range(X.shape[0]):
        result+=(alpha[i]*Y[i]*(X[i,:] , x,sigma));

    result+=b

    return result

In [23]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [24]:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped

In [25]:
def computeW(alphas, X, Y):
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*Y[i],X[i,:].T)
        return w

In [26]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))
    
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [27]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            
            prev_cost = cost
            nth += 1
    return weights

In [28]:
def init3():
    print("reading dataset...")
    # read data in pandas (pd) data frame
    data = pd.read_csv('Wholesaledata.csv')
    print(data.head())

    # drop last column (extra column added by pd)
    # and unnecessary first column (id)
    data.drop(data.columns[[0]], axis=1, inplace=True)
    
    #Mapping values of target variable quality to 'low', 'medium' and 'high' categories for classification
    data['Region']=data['Region'].map({1:'low',2:'medium',3:'high'})  
    #df['quality']=df['quality'].map({'low':0,'medium':1,'high':2})
    
    
    diag_map = {'low': 1.0, 'medium': -1.0,'high':-1.0}
    data['Region'] = data['Region'].map(diag_map)
    
    
    
    # put features & outputs in different data frames
    Y = data.loc[:, 'Region']
    X = data.iloc[:, 0:-1]
    # filter features
    #remove_correlated_features(X)
    #Xnew = np.apply_along_axis(new_features, 1, X)
    #remove_less_significant_features(X, Y)
    # normalize data for better convergence and to prevent overflow
    #X_normalized = MinMaxScaler().fit_transform(Xnew)
    #X = pd.DataFrame(X_normalized)

    # insert 1 in every row for intercept b
    #X.insert(loc=len(X.columns), column='intercept', value=1)

    # split data into train and test set
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)
    # train the model
    print("training started...")
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("training finished.")
    print("weights are: {}".format(W))
    print(W.shape)
    # testing the model
    print("testing the model...")
    y_train_predicted = np.array([])
    for i in range(X_train.shape[0]):
        yp = np.sign(np.dot(X_train.to_numpy()[i], W))
        y_train_predicted = np.append(y_train_predicted, yp)

    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(X_test.to_numpy()[i], W))
        y_test_predicted = np.append(y_test_predicted, yp)
    # An unknow data given by me to check which class it belongs to
    #a = np.array([0.8, 0.6, 0.75, 0.75, 1])
    #yp = np.sign(np.dot(a, W))
    #print("Y value for a : ",yp)
    print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))

In [29]:
regularization_strength = 1000
learning_rate = 0.001

In [30]:
import time
start = time.time()
init3()
end = time.time()
print("TIME TAKEN: {} \n".format(end-start))

reading dataset...
   Channel  Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen
0        2       3  12669  9656     7561     214              2674        1338
1        2       3   7057  9810     9568    1762              3293        1776
2        2       3   6353  8808     7684    2405              3516        7844
3        1       3  13265  1196     4221    6404               507        1788
4        2       3  22615  5410     7198    3915              1777        5185
splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 29815034026.192158
Epoch is: 2 and Cost is: 29926506943.72587
Epoch is: 4 and Cost is: 68243382974.069305
Epoch is: 8 and Cost is: 28205400052.650566
Epoch is: 16 and Cost is: 32540448512.04837
Epoch is: 32 and Cost is: 117869379986.00006
Epoch is: 64 and Cost is: 86056396183.03453
Epoch is: 128 and Cost is: 20092450820.406536
Epoch is: 256 and Cost is: 157181661057.41254
Epoch is: 512 and Cost is: 47943140835.52

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

In [2]:
df_train = pd.read_csv('Wholesaledata.csv')

In [3]:
df_train

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867
438,1,3,10290,1981,2232,1038,168,2125


In [4]:
df_lable1 = df_train["Region"]

In [5]:
df_lable1

0      3
1      3
2      3
3      3
4      3
      ..
435    3
436    3
437    3
438    3
439    3
Name: Region, Length: 440, dtype: int64

In [6]:
df_train.drop('Region',axis=1,inplace=True)

In [7]:
df_train

,Channel,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,12669,9656,7561,214,2674,1338
1,2,7057,9810,9568,1762,3293,1776
2,2,6353,8808,7684,2405,3516,7844
3,1,13265,1196,4221,6404,507,1788
4,2,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...
435,1,29703,12051,16027,13135,182,2204
436,1,39228,1431,764,4510,93,2346
437,2,14531,15488,30243,437,14841,1867
438,1,10290,1981,2232,1038,168,2125


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(df_train,df_lable1,test_size = 0.2)

In [10]:
model = SVC(kernel = 'rbf')
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
import time
start = time.time()

print(model.score(X_test,y_test))
end = time.time()
print("TIME TAKEN: {} \n".format(end-start))

0.75
TIME TAKEN: 0.001993894577026367 



In [31]:
import time
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

In [32]:
def lvec(a):
    n = a.shape[0]
    s = np.empty(0)
    for i in range(n):
        temp = a[i]*a[i:]
        temp[0] = temp[0]/2
        s = np.append(s,temp)
    return s

In [33]:
def create_z(x):
    s = lvec(x)
    z = np.empty(0)
    z = np.append(z,s)
    z = np.append(z,x)
    z = np.append(z,1)
    return z

In [34]:
def create_r(x):
    z = create_z(x)
    eta = lvec(z)
    s = lvec(x)
    r = np.empty(0)
    r = np.append(r,eta)
    r = np.append(r,s)
    return r

In [35]:
def new_feature(x):
    r = create_r(x)
    xnew = np.empty(0)
    xnew = np.append(xnew,r)
    xnew = np.append(xnew,x)
    xnew = np.append(xnew,1)
    return xnew

In [36]:
def new_feature(x):
    r = create_r(x)
    xnew = np.empty(0)
    xnew = np.append(xnew,r)
    xnew = np.append(xnew,x)
    xnew = np.append(xnew,1)
    return xnew

In [37]:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped


def remove_less_significant_features(X, Y):
    sl = 0.05
    regression_ols = None
    columns_dropped = np.array([])
    for itr in range(0, len(X.columns)):
        regression_ols = sm.OLS(Y, X).fit()
        max_col = regression_ols.pvalues.idxmax()
        max_val = regression_ols.pvalues.max()
        if max_val > sl:
            X.drop(max_col, axis='columns', inplace=True)
            columns_dropped = np.append(columns_dropped, [max_col])
        else:
            break
    regression_ols.summary()
    return columns_dropped

In [38]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)

    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [39]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [40]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [41]:
def init3():
    print("reading dataset...")
    # read data in pandas (pd) data frame
    data = pd.read_csv('Wholesaledata.csv')
    print(data.head())

    # drop last column (extra column added by pd)
    # and unnecessary first column (id)
    data.drop(data.columns[[0]], axis=1, inplace=True)
    
    #Mapping values of target variable quality to 'low', 'medium' and 'high' categories for classification
    data['Region']=data['Region'].map({1:'low',2:'medium',3:'high'})  
    #df['quality']=df['quality'].map({'low':0,'medium':1,'high':2})
    
    
    diag_map = {'low': 1.0, 'medium': -1.0,'high':-1.0}
    data['Region'] = data['Region'].map(diag_map)
    
    
    
    # put features & outputs in different data frames
    Y = data.loc[:, 'Region']
    X = data.iloc[:, 0:-1]
    # filter features
    #remove_correlated_features(X)
    #Xnew = np.apply_along_axis(new_features, 1, X)
    #remove_less_significant_features(X, Y)
    # normalize data for better convergence and to prevent overflow
    #X_normalized = MinMaxScaler().fit_transform(Xnew)
    #X = pd.DataFrame(X_normalized)

    # insert 1 in every row for intercept b
    #X.insert(loc=len(X.columns), column='intercept', value=1)

    # split data into train and test set
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)
    # train the model
    print("training started...")
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("training finished.")
    print("weights are: {}".format(W))
    print(W.shape)
    # testing the model
    print("testing the model...")
    y_train_predicted = np.array([])
    for i in range(X_train.shape[0]):
        yp = np.sign(np.dot(X_train.to_numpy()[i], W))
        y_train_predicted = np.append(y_train_predicted, yp)

    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(X_test.to_numpy()[i], W))
        y_test_predicted = np.append(y_test_predicted, yp)
    # An unknow data given by me to check which class it belongs to
    #a = np.array([0.8, 0.6, 0.75, 0.75, 1])
    #yp = np.sign(np.dot(a, W))
    #print("Y value for a : ",yp)
    print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))

In [42]:
regularization_strength = 1000
learning_rate = 0.001

In [43]:
import time
start = time.time()
init3()
end = time.time()
print("TIME TAKEN: {} \n".format(end-start))

reading dataset...
   Channel  Region  Fresh  Milk  Grocery  Frozen  Detergents_Paper  Delicassen
0        2       3  12669  9656     7561     214              2674        1338
1        2       3   7057  9810     9568    1762              3293        1776
2        2       3   6353  8808     7684    2405              3516        7844
3        1       3  13265  1196     4221    6404               507        1788
4        2       3  22615  5410     7198    3915              1777        5185
splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 27506365599.993946
Epoch is: 2 and Cost is: 36928072596.547455
Epoch is: 4 and Cost is: 52024208353.970055
Epoch is: 8 and Cost is: 247915089345.0775
Epoch is: 16 and Cost is: 17342624572.93755
Epoch is: 32 and Cost is: 105125012405.60649
Epoch is: 64 and Cost is: 82133652228.47559
Epoch is: 128 and Cost is: 51936016797.39778
Epoch is: 256 and Cost is: 40059928047.72598
Epoch is: 512 and Cost is: 111461035763.886